In [165]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_tweedie_deviance

In [166]:
data = pd.read_csv('/Users/arthurrastelli/Desktop/Assignment/Assignment.csv')
data = data.rename(columns= {'duree': 'expo','nbrtotan': 'freq','nbrtotc': 'nclaims'})
data["sev"] = data['chargtot']/data['nclaims']
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163657 entries, 0 to 163656
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   AGEPH     163657 non-null  int64  
 1   CODPOSS   163657 non-null  int64  
 2   expo      163657 non-null  float64
 3   lnexpo    163657 non-null  float64
 4   nclaims   163657 non-null  int64  
 5   freq      163657 non-null  float64
 6   chargtot  163657 non-null  float64
 7   agecar    163657 non-null  object 
 8   sexp      163657 non-null  object 
 9   fuelc     163657 non-null  object 
 10  split     163657 non-null  object 
 11  usec      163657 non-null  object 
 12  fleetc    163657 non-null  object 
 13  sportc    163657 non-null  object 
 14  coverp    163657 non-null  object 
 15  powerc    163657 non-null  object 
 16  sev       18345 non-null   float64
dtypes: float64(5), int64(3), object(9)
memory usage: 21.2+ MB


,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,NaN
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,NaN
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,NaN
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,NaN


Bin CODPOSS

In [167]:
def categorize_postal_code(code):
    if 1000 <= code <= 1299:
        return 'BHG'
    elif 1300 <= code <= 1499:
        return 'WB'
    elif 1500 <= code <= 1999 or 3000 <= code <= 3499:
        return 'VB'
    elif 2000 <= code <= 2999:
        return 'ANT'
    elif 3500 <= code <= 3999:
        return 'LIM'
    elif 4000 <= code <= 4999:
        return 'LUI'
    elif 5000 <= code <= 5999:
        return 'NAM'
    elif 6000 <= code <= 6599 or 7000 <= code <= 7999:
        return 'HEN'
    elif 6600 <= code <= 6999:
        return 'LUX'
    elif 8000 <= code <= 8999:
        return 'WV'
    elif 9000 <= code <= 9999:
        return 'OV'
    else:
        return 'Unknown'
    
data['region'] = data['CODPOSS'].apply(categorize_postal_code)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,region
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,NaN,BHG
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,BHG
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,NaN,BHG
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,NaN,BHG
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,NaN,BHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163652,50,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Once,Private,No,No,MTPL+,<66,NaN,OV
163653,86,9960,1.000000,0.000000,0,0.000000,0.000000,6-10,Male,Petrol,Once,Private,No,No,MTPL,<66,NaN,OV
163654,64,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Twice,Private,No,No,MTPL,<66,NaN,OV
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,OV


GENERALIZED LINEAR MODELS: FREQUENCY

In [168]:
bins = [19,29,39,49,59,69,79,89,99]
labels=['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-99']

data['age_bins'] = pd.cut(data['AGEPH'],bins=bins,labels=labels,right=False)

In [169]:
formula1 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [170]:
glm_fit1 = smf.glm(formula=formula1, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit1.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               163638
Model:                            GLM   Df Residuals:                   163607
Model Family:                 Poisson   Df Model:                           30
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -91512.
Date:                Mon, 20 May 2024   Deviance:                   1.4332e+05
Time:                        21:19:09   Pearson chi2:                 5.66e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03051
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

REMOVE THE USEC VARIABLE AS IT IS INSIGNIFICANT

In [20]:
formula2 = '''
freq ~
AGEPH + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [21]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               163657
Model:                            GLM   Df Residuals:                   163633
Model Family:                 Poisson   Df Model:                           23
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -91817.
Date:                Mon, 20 May 2024   Deviance:                   1.4392e+05
Time:                        19:14:21   Pearson chi2:                 5.97e+07
No. Iterations:                     7   Pseudo R-squ. (CS):            0.02749
Covariance Type:            nonrobust                                         
                                                        coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------

BIN THE AGEPH variable

In [22]:
bins = [19,29,39,49,59,69,79,89,99]
labels=['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-99']

data['age_bins'] = pd.cut(data['AGEPH'],bins=bins,labels=labels,right=False)

In [23]:
formula3 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [24]:
glm_fit3 = smf.glm(formula=formula3, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit3.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               163638
Model:                            GLM   Df Residuals:                   163608
Model Family:                 Poisson   Df Model:                           29
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -91513.
Date:                Mon, 20 May 2024   Deviance:                   1.4332e+05
Time:                        19:14:24   Pearson chi2:                 5.65e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03051
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

REMOVE THE INSIGNIFICANT VARIABLES
1. agecat 70-79

In [25]:
data = data[data['age_bins'] != '70-79']

In [26]:
formula4 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [27]:
glm_fit4 = smf.glm(formula=formula4, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit4.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               161551
Model:                            GLM   Df Residuals:                   161522
Model Family:                 Poisson   Df Model:                           28
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -90314.
Date:                Mon, 20 May 2024   Deviance:                   1.4139e+05
Time:                        19:14:28   Pearson chi2:                 5.63e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03091
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

REMOVE INSIGNIFICANT VARIABLES
2. WB

In [28]:
data = data[data['region']!="WB"]

In [29]:
formula5 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [30]:
glm_fit5 = smf.glm(formula=formula5, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit5.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               154947
Model:                            GLM   Df Residuals:                   154919
Model Family:                 Poisson   Df Model:                           27
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -86910.
Date:                Mon, 20 May 2024   Deviance:                   1.3626e+05
Time:                        19:14:31   Pearson chi2:                 5.63e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03143
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

REMOVE INSIGNIFICANT VARIABLES: 
3. LUI

In [31]:
data = data[data['region']!="LUI"]

In [32]:
formula6 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [33]:
glm_fit6 = smf.glm(formula=formula6, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit6.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               133532
Model:                            GLM   Df Residuals:                   133505
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -76581.
Date:                Mon, 20 May 2024   Deviance:                   1.2081e+05
Time:                        19:14:33   Pearson chi2:                 5.55e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.03346
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

REMOVE INSIGNIFICANT VARIABLES

In [34]:
data = data[data['region']!="ANT"]

In [35]:
formula7 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [36]:
glm_fit7 = smf.glm(formula=formula7, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit7.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               113979
Model:                            GLM   Df Residuals:                   113953
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -65745.
Date:                Mon, 20 May 2024   Deviance:                   1.0369e+05
Time:                        19:14:36   Pearson chi2:                 3.89e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.03756
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

REMOVE THE >110 In POWERC

In [37]:
data = data[data['powerc']!=">110"]

In [38]:
formula8 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [39]:
glm_fit8 = smf.glm(formula=formula8, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit8.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               112849
Model:                            GLM   Df Residuals:                   112824
Model Family:                 Poisson   Df Model:                           24
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -65230.
Date:                Mon, 20 May 2024   Deviance:                   1.0293e+05
Time:                        19:14:38   Pearson chi2:                 3.89e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.03773
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [49]:
prediction = glm_fit8.predict(data)
diff = data['freq']-prediction
diff = diff.dropna()
diff
rmse = np.mean(diff**2)
rmse

1.0997902339595176

GENERALIZED LINEAR MODEL

In [156]:
data = pd.read_csv('/Users/arthurrastelli/Desktop/Assignment/Assignment.csv')
data = data.rename(columns= {'duree': 'expo','nbrtotan': 'freq','nbrtotc': 'nclaims'})
data = data[data['nclaims'] != 0]
data["sev"] = data['chargtot']/data['nclaims']
data["logsev"] = np.log(data["sev"])
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 18345 entries, 1 to 163656
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AGEPH     18345 non-null  int64  
 1   CODPOSS   18345 non-null  int64  
 2   expo      18345 non-null  float64
 3   lnexpo    18345 non-null  float64
 4   nclaims   18345 non-null  int64  
 5   freq      18345 non-null  float64
 6   chargtot  18345 non-null  float64
 7   agecar    18345 non-null  object 
 8   sexp      18345 non-null  object 
 9   fuelc     18345 non-null  object 
 10  split     18345 non-null  object 
 11  usec      18345 non-null  object 
 12  fleetc    18345 non-null  object 
 13  sportc    18345 non-null  object 
 14  coverp    18345 non-null  object 
 15  powerc    18345 non-null  object 
 16  sev       18345 non-null  float64
 17  logsev    18345 non-null  float64
dtypes: float64(6), int64(3), object(9)
memory usage: 2.7+ MB


,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290


Bin the CODPOSS

In [157]:
def categorize_postal_code(code):
    if 1000 <= code <= 1299:
        return 'BHG'
    elif 1300 <= code <= 1499:
        return 'WB'
    elif 1500 <= code <= 1999 or 3000 <= code <= 3499:
        return 'VB'
    elif 2000 <= code <= 2999:
        return 'ANT'
    elif 3500 <= code <= 3999:
        return 'LIM'
    elif 4000 <= code <= 4999:
        return 'LUI'
    elif 5000 <= code <= 5999:
        return 'NAM'
    elif 6000 <= code <= 6599 or 7000 <= code <= 7999:
        return 'HEN'
    elif 6600 <= code <= 6999:
        return 'LUX'
    elif 8000 <= code <= 8999:
        return 'WV'
    elif 9000 <= code <= 9999:
        return 'OV'
    else:
        return 'Unknown'
    
data['region'] = data['CODPOSS'].apply(categorize_postal_code)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev,region
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693,BHG
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895,BHG
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723,BHG
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552,BHG
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290,BHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163624,43,9700,1.000000,0.000000,2,2.000000,4270.511330,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,2135.255665,7.666342,OV
163636,58,9820,1.000000,0.000000,1,1.000000,145.017712,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,145.017712,4.976856,OV
163642,27,9890,1.000000,0.000000,1,1.000000,271.220305,2-5,Male,Petrol,Once,Private,No,No,MTPL+++,<66,271.220305,5.602931,OV
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,8.064006,OV


BIN THE AGEPH VARIABLE

In [158]:
bins = [19,29,39,49,59,69,79,89,99]
labels=['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-99']

data['age_bins'] = pd.cut(data['AGEPH'],bins=bins,labels=labels,right=False)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev,region,age_bins
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693,BHG,0-19
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895,BHG,60-69
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723,BHG,0-19
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552,BHG,40-49
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290,BHG,40-49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163624,43,9700,1.000000,0.000000,2,2.000000,4270.511330,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,2135.255665,7.666342,OV,30-39
163636,58,9820,1.000000,0.000000,1,1.000000,145.017712,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,145.017712,4.976856,OV,40-49
163642,27,9890,1.000000,0.000000,1,1.000000,271.220305,2-5,Male,Petrol,Once,Private,No,No,MTPL+++,<66,271.220305,5.602931,OV,0-19
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,8.064006,OV,40-49


In [159]:
formula_sev1 = '''
sev ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [160]:
glm_fit_sev1 = smf.glm(formula=formula_sev1, data=data, family = sm.families.Gamma(link=sm.families.links.Log())).fit()
print(glm_fit_sev1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    sev   No. Observations:                18340
Model:                            GLM   Df Residuals:                    18309
Model Family:                   Gamma   Df Model:                           30
Link Function:                    Log   Scale:                          40.323
Method:                          IRLS   Log-Likelihood:            -1.8245e+05
Date:                Mon, 20 May 2024   Deviance:                       44964.
Time:                        21:10:37   Pearson chi2:                 7.38e+05
No. Iterations:                    49   Pseudo R-squ. (CS):           0.002524
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [164]:
predictions = glm_fit_sev1.predict()
diff = data['freq']
mse = np.mean(diff)
mse

1.4057405764903028